In [1]:
import pandas as pd
df = pd.read_pickle('yonhap_ready.pkl')

In [2]:
df.head(3)

,article,title,article_len,sec1,sec_idx
0,'회장이 청부폭력' 루머 협박범도 기소…사이버명예훼손 전담수사 첫 사례 ...,"""해경이 가만있으라 방송"" 허위사실 유포 40대 기소",966,사회,4
1,지난 해 9월 북한의 함경북도 항구도시 나진과 러시아 극동지역 도시 ...,南 점검단 방북…나진-하산 프로젝트 시범운송 시작(종합),1539,정치,6
2,포항 도착한 화물선 러시아 시베리아산 유연탄 4만 500t을...,나진-하산 프로젝트 탄력…시범수송 석탄 포항 도착(종합),1827,경제,0


In [3]:
X = df.article
y = df.sec_idx
print(X.shape)
print(y.shape)

(443312,)
(443312,)


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(376815,)
(66497,)
(376815,)
(66497,)


In [5]:
%%time
from konlpy.tag import Mecab; m = Mecab()
#pos = lambda d: ['/'.join(p) for p in m.pos(d)]
def noun_tagger(text):
    #return [pos[0] for pos in m.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    return [pos[0] for pos in m.pos(text) if pos[1].startswith('NN')]

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 24.1 ms


### Summary:
    * vect.fit(train) learns the vocabulary of the training data
    * vect.transform(train) uses the fitted vocabulary to build a document term matrix from the training data
    * vect.transform(test) uses the fitted vocabulary to build a document term matrix from the testing data(and ignores tokens it hasn't seen before)

In [6]:
stopwords_news=['지난해', '중','말', '뒤', '곳', '군', '위', '개', '간', '건', '이날', '도', '등', '명', '시', '앞', '원', '분', '회', '년', '것', '씨', '일', '월','오전','오후']

In [7]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(sublinear_tf=True, tokenizer=noun_tagger,stop_words=stopwords_news, ngram_range=(1,3), min_df=5, max_df=.5)
X_train_dtm = vect.fit_transform(X_train)

CPU times: user 24min 5s, sys: 18.9 s, total: 24min 24s
Wall time: 24min 23s


In [8]:
X_test_dtm = vect.transform(X_test)

In [9]:
X_train_dtm.shape

(376815, 2473920)

In [10]:
X_test_dtm.shape

(66497, 2473920)

## save vector & dtm

https://stackoverflow.com/questions/29788047/keep-tfidf-result-for-predicting-new-content-using-scikit-for-python
https://stackoverflow.com/questions/32764991/how-do-i-store-a-tfidfvectorizer-for-future-use-in-scikit-learn

In [11]:
import pickle

with open("yonhap_tfidf_vect.pkl", 'wb') as handle:
    pickle.dump(vect, handle)

with open("yonhap_dtm.pkl", 'wb') as handle:
    pickle.dump(X_test_dtm, handle)

In [16]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=.001)

In [17]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 3.06 s, sys: 176 ms, total: 3.24 s
Wall time: 3.24 s


MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)

In [18]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [19]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.86871588191948512

In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_class, target_names=['경제', '과학', '국제', '문화', '사회', '생활', '정치']))

             precision    recall  f1-score   support

         경제       0.93      0.90      0.92     16445
         과학       0.47      0.79      0.59       362
         국제       0.85      0.90      0.87     12919
         문화       0.76      0.90      0.82      3605
         사회       0.88      0.83      0.85     17451
         생활       0.50      0.76      0.61      1237
         정치       0.91      0.86      0.88     14478

avg / total       0.88      0.87      0.87     66497



In practice Multinomial NB can be better than Linear SVM in some situations and svm better than NB on other datasets. Apparently t's possible to combine the two approaches to get a very good baseline: projects:nbsvm - Sida I. Wang

You can also try non-linear SVMs but often the quadratic complexity of the SMO algorithm (for instance as implemented in libsvm) makes it not practical on datasets with more than 5000 documents. Instead on prefer to use liblinear than can only train linear SVM on large datasets.

https://www.quora.com/What-are-the-differences-similarities-between-SVM-Naive-Bayes-for-binary-text-classification-wrt-how-they-are-processing-the-features

http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

In [21]:
from sklearn import svm
#clf = svm.SVC(decision_function_shape='ovo')
clfrSVM=svm.LinearSVC(C=1)

In [22]:
%time clfrSVM.fit(X_train_dtm, y_train)

CPU times: user 1min 28s, sys: 704 ms, total: 1min 29s
Wall time: 1min 29s


LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [23]:
y_pred_class_svm = clfrSVM.predict(X_test_dtm)

In [24]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class_svm)

0.91334947441237946

In [25]:
print(classification_report(y_test, y_pred_class_svm, target_names=['경제', '과학', '국제', '문화', '사회', '생활', '정치']))

             precision    recall  f1-score   support

         경제       0.95      0.94      0.94     16445
         과학       0.76      0.65      0.70       362
         국제       0.89      0.93      0.91     12919
         문화       0.88      0.88      0.88      3605
         사회       0.90      0.91      0.91     17451
         생활       0.79      0.62      0.70      1237
         정치       0.93      0.91      0.92     14478

avg / total       0.91      0.91      0.91     66497



https://stackoverflow.com/questions/40115043/no-space-left-on-device-error-while-fitting-sklearn-model

In [26]:
import numpy as np
counts = np.bincount(y_pred_class[(y_pred_class!=2) & (y_test==2)])
print(np.argmax(counts))

0


After removing short articles to improve accuracy, the accuracy is still around 91% when I use SVM. The main cause can be found belown example. Below articles shows why so many science news articles are predicted as a economics articles. Those articles looks economics articles. I think this happens because the category have chosen by reporters. So mainly due to data error.

In [27]:
X_test[(y_pred_class==0) & (y_test==2)]

134870           아시아인프라투자은행(AIIB) 개소식에서 연설하는 리커창 중국 총리  ...
237177           15,000 무너진 일본 주가  소비위축에 대외악재 산적…참의원 선거 ...
184399              1년만에 만장일치 결정 "노동시장 호조·물가상승 고려한 조치, 경...
314134           국제유가는 10일(현지시간) 비교적 큰 폭의 오름세를 보였다.     ...
29999            LG디스플레이, 베이징서 '아트슬림 프로모션' 행사 개최      LG...
131833           중국 인민은행       중국 당국의 유동성 투입에도 춘제(春節·음력설...
59946                그리스 재무부가 11일(현지시간) 국제통화기금(IMF) 채무 7...
31614            중국인, 미국 내 외국인 주택 거래 '큰손'으로       지난해 미국...
196063           런던 남동부의 주택가   "세 사는 가구 늘고 임차료도 올라 저축은커녕...
220499           우버 홈페이지 캡처  '모든 것 위한 우버'로 영역확장…'문어발 확장'...
155731            일본의 국가 채무가 사상 최고치를 경신했다.      일본 재무성은 ...
129654           한적한 중국 베이징의 한 백화점       중국의 지난해 재정수입 증가...
406737          스마트폰에 엇갈린 희비…대만은 작년 아이폰7 덕에 GDP 깜짝 성장   ...
46483            지우마 호세프 브라질 대통령       브라질 경제가 원자재 가격 하락...
217240           일본 주식 시장   외국인투자 증가 주요인…25년 연속 세계 최대 채권...
176502           중국 창업의 산실 중관춘［EPA=연합뉴스 자료사진］       중국에서...
94875                세계 최대 원유수입국인 중국이 저장

In [28]:
df[df.index==247480]

,article,title,article_len,sec1,sec_idx
247480,인공지능 '알파고'가 이세돌 9단에 3연승하자 중국 일부 ...,"<세기의 대국> 중 누리꾼, 중국판 알파고 '마이고'에 기대감",648,국제,2


In [29]:
with open("yonhap_svm_clf.pkl", 'wb') as handle:
    pickle.dump(clfrSVM, handle)

In [30]:
with open("yonhap_nb_clf.pkl", 'wb') as handle:
    pickle.dump(nb, handle)